In [1]:
import elasticsearch
from elasticsearch_dsl import Search
import scipy.sparse as sparse
from sklearn.naive_bayes import GaussianNB

In [2]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn import tree
import numpy as np

In [3]:
es = elasticsearch.Elasticsearch()

In [4]:
results = elasticsearch.helpers.scan(es,
    index="spam_ham_2",
    doc_type="doc",
    preserve_order=True,
    query={"query": {"match_all": {}}},
)

index = 0
test_index = 0

train_doc_id_mapping = {}
train_id_doc_mapping = {}
train_is_spam = []


test_doc_id_mapping = {}
test_id_doc_mapping = {}
test_is_spam = []


for item in results:
    
    if item['_source']['train'] == "True":    
        train_doc_id_mapping[item['_id']] = index
        train_id_doc_mapping[index] = item['_id']
        is_spam = 0

        if item['_source']['spam'] == "True":
            is_spam = 1
            
        train_is_spam.append(is_spam)
        index = index + 1
        
    else:
        
           
        test_doc_id_mapping[item['_id']] = test_index
        test_id_doc_mapping[test_index] = item['_id']
        is_spam = 0

        if item['_source']['spam'] == "True":
            is_spam = 1
            
        test_is_spam.append(is_spam)
        test_index = test_index + 1
        

In [5]:
len(train_doc_id_mapping),len(test_doc_id_mapping), len(train_is_spam), len(test_is_spam)

(60336, 15083, 60336, 15083)

In [6]:
with open("features_provided") as file:
    features = file.read().split(" ")[:-1]

In [7]:
feature_id_mapping = {}
id_feature_mapping = {}
index = 0

for word in features:
    feature_id_mapping[word.lower()] = index
    id_feature_mapping[index] = word.lower()
    index = index + 1
    

In [8]:
# body_doc

In [9]:
docs_analysed = {}
body_doc = {
    "query": {    
        "match" : {
            "text" : ""
            }
        },
    "from" : 0,
    "size" : 1000
}

for i in feature_id_mapping:
    print(i , end=", ")
    body_doc["query"]["match"]["text"] = i
    total_res = []
    hits = es.search(index="spam_ham_2", body=body_doc)
    total_res = total_res + hits['hits']['hits']
    
    while len(total_res) < hits['hits']['total']:
        body_doc["from"] = len(total_res)
        hits = es.search(index="spam_ham_2", body=body_doc, size=1000)
        total_res = total_res + hits['hits']['hits']
    
    if len(total_res) == 0:
        continue
    else:
        for doc in total_res:
            if doc["_id"] not in train_doc_id_mapping:
                continue
            if doc["_id"] in docs_analysed:
                continue
            docs_analysed[doc["_id"]] = {}
            term_vects = es.termvectors(index = 'spam_ham_2', 
               doc_type = 'doc', 
               id = doc["_id"], 
               fields = ["text"], 
               payloads=False, 
               positions=False,
               offsets=False,
               term_statistics=True,
               field_statistics=True)
            
            for word in features:
                if(word.lower() in term_vects['term_vectors']['text']['terms']):
                    tf = term_vects['term_vectors']['text']['terms'][word.lower()]["term_freq"]
                    docs_analysed[doc["_id"]][feature_id_mapping[word.lower()]] = tf

credit, fast, visit, act, meet, luxury, weight, home, our, collect, remove, ad, order, limited, medicine, refinance, trial, singles, free, click, cash, only, bonus, loans, deal, unsolicited, $discount, here, 100%, performance, valium, interest, join, website, celebrity, profit, price, sales, win, diagnostics, $$$, miracle, spam, extra, dollars, affordable, success, clearance, buy, subscribe, prize, lose, legal, double, earn, rate, check, hidden, shopper, virus, money, chance, incredible, viagra, instant, freedom, malware, compare, million, now, no, lower, 

In [10]:
row = []
col = []
data = []
# label_train = []


# row_test = []
# col_test = []
# data_test = []
# label_test = []

for doc in docs_analysed:
    for term in docs_analysed[doc]:
        if doc in train_doc_id_mapping:
            
            row.append(train_doc_id_mapping[doc])
            col.append(term)
            data.append(docs_analysed[doc][term])

#         else:
#             row_test.append(test_doc_id_mapping[doc])
#             col_test.append(term)
#             data_test.append(docs_analysed[doc][term])
            

In [11]:
sparse_mat_train = sparse.csr_matrix((data, (row, col)), shape=(len(train_doc_id_mapping), len(features)))

In [12]:
sparse_mat_train

<60336x83 sparse matrix of type '<class 'numpy.int64'>'
	with 226724 stored elements in Compressed Sparse Row format>

In [13]:
docs_analysed = {}
body_doc = {
    "query": {    
        "match" : {
            "text" : ""
            }
        },
    "from" : 0,
    "size" : 1000
}


for i in feature_id_mapping:
    print(i , end=", ")
    body_doc["query"]["match"]["text"] = i
    total_res = []
    hits = es.search(index="spam_ham_2", body=body_doc)
    total_res = total_res + hits['hits']['hits']
    
    while len(total_res) < hits['hits']['total']:
        body_doc["from"] = len(total_res)
        hits = es.search(index="spam_ham_2", body=body_doc, size=1000)
        total_res = total_res + hits['hits']['hits']
    
    if len(total_res) == 0:
        continue
    else:
        for doc in total_res:
            if doc["_id"] not in test_doc_id_mapping:
                continue
            if doc["_id"] in docs_analysed:
                continue
            docs_analysed[doc["_id"]] = {}
            term_vects = es.termvectors(index = 'spam_ham_2', 
               doc_type = 'doc', 
               id = doc["_id"], 
               fields = ["text"], 
               payloads=False, 
               positions=False,
               offsets=False,
               term_statistics=True,
               field_statistics=True)
            
            for word in features:
                if(word.lower() in term_vects['term_vectors']['text']['terms']):
                    tf = term_vects['term_vectors']['text']['terms'][word.lower()]["term_freq"]
                    docs_analysed[doc["_id"]][feature_id_mapping[word.lower()]] = tf

credit, fast, visit, act, meet, luxury, weight, home, our, collect, remove, ad, order, limited, medicine, refinance, trial, singles, free, click, cash, only, bonus, loans, deal, unsolicited, $discount, here, 100%, performance, valium, interest, join, website, celebrity, profit, price, sales, win, diagnostics, $$$, miracle, spam, extra, dollars, affordable, success, clearance, buy, subscribe, prize, lose, legal, double, earn, rate, check, hidden, shopper, virus, money, chance, incredible, viagra, instant, freedom, malware, compare, million, now, no, lower, 

In [14]:
row_test = []
col_test = []
data_test = []

for doc in docs_analysed:
    for term in docs_analysed[doc]:
        if doc in test_doc_id_mapping:
            
            row_test.append(test_doc_id_mapping[doc])
            col_test.append(term)
            data_test.append(docs_analysed[doc][term])
            

In [15]:
sparse_mat_test = sparse.csr_matrix((data_test, (row_test, col_test)), shape=(len(test_doc_id_mapping), len(features)))

In [16]:
sparse_mat_train

<60336x83 sparse matrix of type '<class 'numpy.int64'>'
	with 226724 stored elements in Compressed Sparse Row format>

In [17]:
sparse_mat_test

<15083x83 sparse matrix of type '<class 'numpy.int64'>'
	with 56738 stored elements in Compressed Sparse Row format>

In [18]:
clf = tree.DecisionTreeClassifier()
clf.fit(sparse_mat_train, train_is_spam)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [19]:
print(accuracy_score(clf.predict(sparse_mat_train), train_is_spam))

0.987370723945903


In [20]:
test_pred = clf.predict(sparse_mat_test)

In [21]:
print(accuracy_score(test_pred, test_is_spam))

0.9678445932506796


In [22]:
body_doc = {
  "query" :{
    "match": {
      "name": ""
    }
  } 
}

c = 1
for i in np.argsort(test_pred)[::-1][0:50]:
    
    body_doc["query"]["match"]["name"] = "trec07p/data/" + test_id_doc_mapping[i]
    hits = es.search(index="spam_ham_2", body=body_doc)
    print(c,"\t", test_id_doc_mapping[i], "\t","spam:",hits["hits"]["hits"][0]["_source"]["spam"])
    c = c + 1

IndexError: list index out of range